# Calculating accuracies for MobileNet predictions

## 1. Imports

In [106]:
import torch
# Download an example image from the pytorch website
import urllib
from PIL import Image as Image 
from torchvision import transforms
from IPython.display import Image as show_img
import numpy as np
import os
import string
import re
import pandas as pd
import pickle

## 2. Functions

In [128]:
def pre_process(lista, index):
    # remove directories
    lista = [item[index:] for item in lista]
    
    # remove digits
    lista = [re.sub(r'\b\d+\b', '', item) for item in lista]

    # remove hiffens 
    lista = [re.sub(r'\b-\b', ' ', item) for item in lista]

    # remove punctuation
    lista = [item.strip(string.punctuation) for item in lista]
    
    return lista

def get_pil_transform(): 
    transf = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224)
    ])    

    return transf

def get_preprocess_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])     
    transf = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])    

    return transf    

def convert_bars(lista):
    lista = [item.replace('\\', '/') for item in lista]
    return lista

def files_in_folder(path):
    files = []
    for r, d, f in os.walk(path):
        for file in f:
            if '.jpg' in file:
                files.append(os.path.join(r, file))
    files = convert_bars(files)
    return files

def predict(pill_transf, preprocess_transform, image):
    input_tensor = preprocess_transform(pill_transf(input_image))
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    predicted = torch.nn.functional.softmax(output[0], dim=0).argmax()
    
    y_hat = pd.read_csv('labels.csv',index_col = 'id').loc[int(predicted)+1].label
    return y_hat

## 3. Filtering image labels for input images (Caltech 256 images)

In [79]:
directory = "C:/Users/Asus/Documents/IST/MECD 0101/AEP/Projeto/Implementação/Computer Vision/256_ObjectCategories"
dataset_input = [x[0] for x in os.walk(directory)]
dataset_input = pre_process(dataset_input, 105)

df = pd.read_csv('labels.csv',index_col = 'id')
dataset_csv = [df['label'][i] for i in range(len(df))]

filtered_input = [item for item in dataset_input if item in dataset_csv]

print("# Existant classes: {}".format(len(filtered_input)))
filtered_input

# Existant classes: 32


['cannon',
 'canoe',
 'centipede',
 'coffee mug',
 'conch',
 'electric guitar',
 'football helmet',
 'golf ball',
 'goose',
 'harp',
 'hourglass',
 'hummingbird',
 'llama',
 'mushroom',
 'photocopier',
 'rifle',
 'school bus',
 'scorpion',
 'screwdriver',
 'snail',
 'snowmobile',
 'soccer ball',
 'syringe',
 'teapot',
 'tennis ball',
 'toaster',
 'triceratops',
 'trilobite',
 'tripod',
 'umbrella',
 'wine bottle',
 'zebra']

## 4. Load MobileNet model

In [7]:
model = torch.hub.load('pytorch/vision:v0.4.2', 'mobilenet_v2', pretrained=True)
model.eval()

Using cache found in C:\Users\Asus/.cache\torch\hub\pytorch_vision_v0.4.2


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

## 5. Calculating predictions of input images

In [102]:
directory = "C:/Users/Asus/Documents/IST/MECD 0101/AEP/Projeto/Implementação/Computer Vision/32 categorias"
foldernames = [x[0] for x in os.walk(directory)]
foldernames.pop(0)
foldernames = convert_bars(foldernames)

pill_transf = get_pil_transform()
preprocess_transform = get_preprocess_transform()

y_true = []
y_hat = []

try:
    y_true, y_hat = pickle.load(open("DATA LIME PREDICTIONS.pickle","rb"))

except:
    for folder, name in zip(foldernames, filtered_input):
        nested_ytrue = []
        nested_yhat = []
        files_names = files_in_folder(folder)
        for filename in files_names:
            nested_ytrue.append(name)
            with open(filename, "r") as file:
                input_image = Image.open(filename)

                try: 
                    nested_yhat.append(predict(pill_transf, preprocess_transform, input_image))

                except RuntimeError: # black and white image
                    continue
        y_true.append(nested_ytrue)
        y_hat.append(nested_yhat)
    
    data = (y_true, y_hat)
    f = open('DATA LIME PREDICTIONS.pickle', 'wb')
    pickle.dump(data, f)
    f.close()

mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label
mudou de label


## 6. Calculating accuracies for each label

In [137]:
accuracies = np.array([])

for i in range(len(y_true)):
    hits = sum(correct == prediction for correct, prediction in zip(y_true[i], y_hat[i]))
    accuracies = np.append(accuracies, round(hits/len(y_true[i]),3))

## 7. Dataframe with predictions

In [138]:
indexes = []
truelabels = []
predlabels = []
all_accuracies = []

index = 0

for i in range(len(y_true)):
    index = 0
    for correct, prediction in zip(y_true[i], y_hat[i]):
        index += 1
        indexes.append(index)
        truelabels.append(correct)
        predlabels.append(prediction)
        all_accuracies.append(accuracies[i])
        
dfout = pd.DataFrame({"Index of image": indexes,
                       "True label": truelabels,
                       "Prediction label": predlabels,
                       "Accuracy of label": all_accuracies
                      })
dfout = dfout.sort_values(by=['Accuracy of label', 'Index of image'])
path = 'C:/Users/Asus/Documents/IST/MECD 0101/AEP/Projeto/Implementação/Computer Vision'
dfout.to_excel(path+'/Accuracies.xlsx', index=False)
print("File saved successfully.")

File saved successfully.
